In [0]:
# -*- ClusterGenerator.py -*-
"""
Created Feb 2019

@author: Elena Gelzintye / Timothy E H Allen

Clusters dataset into equal parts with similarity less than x between the clusters
"""

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import random
from rdkit import Chem
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform 
from collections import Counter
from rdkit.Chem import rdMolDescriptors

def get_fingerprint(smiles):
    '''generates fingerprint dataframe given the smiles dataframe with 'SMILES' column containing the smiles. Also saves bit information needed for recovering the substructures later'''
    
    bit_infos=[]
    rdkit_molecules=[Chem.MolFromSmiles(x) for x in smiles['SMILES']]
    rdkit_fingerprint=[]
    for mol in rdkit_molecules:
        bit_info={}
        fp=rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=4, nBits=10000, \
                                                                      bitInfo=bit_info).ToBitString() 
        bit_infos.append(bit_info)
        rdkit_fingerprint.append(fp)        
    fingerprint_df=pd.DataFrame([np.array(list(x)).astype(int) for x in rdkit_fingerprint])
    return fingerprint_df, bit_infos

def Tanimoto (fp1, fp2):
    if len(fp1)!=len(fp2):
        print('fingerprint lengths do not match')
        return None
    added=np.add(fp1,fp2)
    double_on=np.count_nonzero(added==2)           
    total_on=np.count_nonzero(added)
    TMS=double_on/total_on
    return TMS
    
def get_similarities(dataframe):
    
#    returns 2d numpy similarity array given the dataframe of fingerprints
        
    similarities=np.zeros((dataframe.shape[0], dataframe.shape[0]))
    fingerprints=np.array(dataframe)
    
    for no_1 in range(0, len(fingerprints)):
        for no_2 in range(no_1, len(fingerprints)):
            
            similarities[no_1, no_2]=Tanimoto(fingerprints[no_1], fingerprints[no_2])

    similarities=similarities+similarities.T-np.diag(similarities.diagonal())
    
    return similarities

class data_point:
    def __init__(self, name, position, cluster=None, cluster_size=None):
        self.name=name
        self.position=position
        self.cluster=cluster
        self.cluster_size=cluster_size
    def print_stuff(self):
        print('{}: {}'.format(self.name, self.position))

receptor = "AR"
smiles=pd.read_csv("/content/drive/My Drive/" + receptor + ".csv".format(receptor,receptor))

#generate fingeprint 
fingerprint, bit_infos=get_fingerprint(smiles) 
fingerprint=np.array(fingerprint)
smiles=np.array(smiles)

#calculate tanimoto similarity matrix
similarities=get_similarities(fingerprint)


#generate distance matrix 
distances=np.zeros((len(similarities), len(similarities)))
for no1 in range(len(similarities)):
    for no2 in range(len(similarities)):
        distances[no1][no2]=1-similarities[no1][no2]
        

#Do the hierarchical clustering
#Hierarchy-cluster into clusters and set cut-off 

distances_condensed=squareform(distances)
Z=linkage(distances_condensed, method='single')

#%%
#for 5-fold cross validation
no_clusters=5
#arbitrarily set the largest cluster size and maximum distance between two clusters (yields more than 5 clusters)
largest_cluster=len(fingerprint)
max_d=0.3
#iterate until the largest cluster is no larger than half of the cross-validation group by allowing increasingly lower max distance between clusters/higher maximum similarity between the clusters. 
while largest_cluster>int(len(fingerprint)/(no_clusters*2)):
    
    max_d=max_d*0.99
    
    clusters = fcluster(Z, max_d, criterion='distance')
    clusters=list(clusters)
    largest_cluster=Counter(clusters).most_common(1)[0][1]
    print(max_d, largest_cluster)
    


counted=Counter(clusters)
#make up a class for each compound/point including its id (here 'no') in the smiles list, fingerprint, cluster_id and the number of compounds in the corresponding cluster. 
all_pts=[data_point(no, fingerprint[no], clust_id, counted[clust_id]) for no, clust_id in enumerate(clusters)]
#sort all points based on the size of the cluster they belong to 
all_pts.sort(key=lambda s: s.cluster_size, reverse=True)

#%%
# group into 5 groups
#initialise with attributing five biggest clusters to different CV groups

#dictionary matching cluster_id to the list of compounds (class) belonging to that cluster
all_clusters={}
for pt in all_pts:
    try:
        all_clusters[pt.cluster].append(pt)
    except KeyError:
        all_clusters[pt.cluster]=[pt]

#get the keys with decreasing cluster size
#get number of points (lengths) and cluster_id in each cluster
lengths=[(len(all_clusters[c]), c) for c in all_clusters]
#sort cluster_ids based on how many compounds are in each
lengths.sort(key= lambda s: s[0], reverse=True)
#get list of cluster ids, sorted from largest to smallest
keys_ordered=[s[1] for s in lengths]

print(keys_ordered)

#ideal size of a 5-CV cluster
ideal_size=(int(len(fingerprint)/5)) + 1

#assign the largest 5 clusters to 5 different cross-validation groups
folds={0:all_clusters[keys_ordered[0]], 
       1:all_clusters[keys_ordered[1]], 
       2:all_clusters[keys_ordered[2]], 
       3:all_clusters[keys_ordered[3]],
       4:all_clusters[keys_ordered[4]]}
#%%
#select remaining cluster ids
keys_ordered=keys_ordered[5:]

keys_left=[]
fold_nos=[0,1,2,3,4]
for key in keys_ordered: 
    
    #select the compounds in selected cluster to append
    to_add=all_clusters[key]
    has_been_added=False
    
    #choose random sequence of CV groups to cycle over
    random.shuffle(fold_nos)
    for fold in fold_nos:
        #add the cluster if there's space in CV group and exit the cycle
        if len(folds[fold])+len(to_add)<ideal_size:
            folds[fold]=folds[fold]+(to_add)
            has_been_added=True
            
            break
    #note which compounds were excluded
    if has_been_added==False:
        keys_left.append(key)
        
fold_a = [a.name for a in folds[0]]
fold_b = [a.name for a in folds[1]]
fold_c = [a.name for a in folds[2]]
fold_d = [a.name for a in folds[3]]
fold_e = [a.name for a in folds[4]]

fingerprint_fold_a = pd.DataFrame(fingerprint[fold_a])
fingerprint_fold_b = pd.DataFrame(fingerprint[fold_b])
fingerprint_fold_c = pd.DataFrame(fingerprint[fold_c])
fingerprint_fold_d = pd.DataFrame(fingerprint[fold_d])
fingerprint_fold_e = pd.DataFrame(fingerprint[fold_e])

smiles_fold_a = pd.DataFrame(smiles[fold_a])
smiles_fold_a = smiles_fold_a.drop([0], axis = 1)
smiles_fold_b = pd.DataFrame(smiles[fold_b])
smiles_fold_b = smiles_fold_b.drop([0], axis = 1)
smiles_fold_c = pd.DataFrame(smiles[fold_c])
smiles_fold_c = smiles_fold_c.drop([0], axis = 1)
smiles_fold_d = pd.DataFrame(smiles[fold_d])
smiles_fold_d = smiles_fold_d.drop([0], axis = 1)
smiles_fold_e = pd.DataFrame(smiles[fold_e])
smiles_fold_e = smiles_fold_e.drop([0], axis = 1)

fingerprint_fold_a = pd.concat([fingerprint_fold_a,smiles_fold_a], axis=1)
fingerprint_fold_b = pd.concat([fingerprint_fold_b,smiles_fold_b], axis=1)
fingerprint_fold_c = pd.concat([fingerprint_fold_c,smiles_fold_c], axis=1)
fingerprint_fold_d = pd.concat([fingerprint_fold_d,smiles_fold_d], axis=1)
fingerprint_fold_e = pd.concat([fingerprint_fold_e,smiles_fold_e], axis=1)

pd.DataFrame(fingerprint_fold_a).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_a fingerprint ECFP4 10000.csv", index = False)
pd.DataFrame(fingerprint_fold_b).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_b fingerprint ECFP4 10000.csv", index = False)
pd.DataFrame(fingerprint_fold_c).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_c fingerprint ECFP4 10000.csv", index = False)
pd.DataFrame(fingerprint_fold_d).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_d fingerprint ECFP4 10000.csv", index = False)
pd.DataFrame(fingerprint_fold_e).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_e fingerprint ECFP4 10000.csv", index = False)

pd.DataFrame(smiles[fold_a]).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_a.csv", index = False)
pd.DataFrame(smiles[fold_b]).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_b.csv", index = False)
pd.DataFrame(smiles[fold_c]).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_c.csv", index = False)
pd.DataFrame(smiles[fold_d]).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_d.csv", index = False)
pd.DataFrame(smiles[fold_e]).to_csv("/content/drive/My Drive/clustered_data/" + receptor +" fold_e.csv", index = False)


        
##extract the compound indices to later be passed on for 
indices=[]
#cycle over 5 combinations of validation + training indices
for k in range(len(folds)):
    
    #a bunch of indices for validation
    #a.name extracts index out of the bundled up data_point class
    val_idx=[a.name for a in folds[k]]
    
    train_idx=[]
    for i in range(len(folds)):
        #excludes the CV group that was selected for validation
        if i!=k:
            train_idx=train_idx+[a.name for a in folds[i]]
            
    indices.append((train_idx,val_idx))
    
print(train_idx)
print(val_idx)

#%% 

indices=np.array(indices)
np.save('{} train test split'.format(receptor), indices)
pd.DataFrame(np.array(indices)).to_csv("AChE train test split.csv")